# Show Certificate Expiration Dates
This notebook will demonstrate the platform APIs for retrieving certificate expiration dates. 

## Setup

Set the `YB_PLATFORM_URL` and `YB_API_KEY` environment variables in a .env file before running this notebook.

Note: "verify=False" is used to disable SSL verification for the purposes of this demonstration, but you should use appropriate certificates in a production environment.


In [ ]:
import os
import requests
from pprint import pprint
from dotenv import load_dotenv
from datetime import datetime, timezone

load_dotenv()

yba_url = os.getenv("YBA_PLATFORM_URL")
yba_api_key = os.getenv("YBA_API_KEY")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{yba_api_key}"
}

session = requests.Session()
session.verify = False
session.headers = headers

pprint(yba_url)
pprint(yba_api_key)
pprint(headers)

## Get the customer ID

The customer ID is unique to the YBA platform and is required for the following Platform API calls. 

In [ ]:
route = f"{yba_url}/api/v1/session_info"
response = session.get(url=route).json()
customer_uuid = response["customerUUID"]

print(customer_uuid)

## List root certificates

List a customer's root certificates along with expiration dates.

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/5a40faca3973c-list-a-customer-s-certificates

In [ ]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/certificates"
response = session.get(url=route)
metrics = response.json()

today = datetime.now(timezone.utc)

for item in metrics:
    start_date = datetime.fromisoformat(item['startDateIso'].replace('Z', '+00:00'))
    expiry_date = datetime.fromisoformat(item['expiryDateIso'].replace('Z', '+00:00'))
    time_to_expiry = expiry_date - today
    
    print(f"{item['label']}\n"
          f"created: {start_date}\n"
          f"expiration: {expiry_date}\n"
          f"time to expiry: {time_to_expiry}\n")

## Show count of expired node-to-node certificates

Example of retrieving a single prometheus metric; in this case, the count of expired node-to-node certificates per universe.

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/732209b7fa9b0-get-prometheus-metrics

In [ ]:
route = f"{yba_url}/api/v1/prometheus_metrics"
response = session.get(url=route)
metrics = response.text

# Search for lines containing your metric
target_metric = "ybp_health_check_n2n_cert"
matching_lines = [line for line in metrics.splitlines() if target_metric in line]
print("\n".join(matching_lines))

## Health check metrics

Example of parsing values from the large health check output. This pulls the node-to-node and client-to-node certificate days remaining before expiration.

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/38a27fa229333-return-results-for-the-last-health-check

In [ ]:
# hard coded; get this programmatically
universe_uuid = "asdfasdf-asdf-asdf-asdf-asdfasdfasdf"

route = f"{yba_url}/api/v1/customers/{customer_uuid}/universes/{universe_uuid}/health_check"
response = session.get(url=route)
metrics = response.json()

# pprint(metrics)

health_checks = [
    (item, metric, value)
    for entry in metrics
    for item in entry.get('data', [])
    for metric in item.get('metrics', [])
    for value in metric.get('values', [])
]
    
n2n_validity_days = {
    int(value['value']) 
    for _, metric, value in health_checks
    if metric['name'] == 'ybp_health_check_n2n_cert_validity_days'
}
    
c2n_validity_days = {
    int(value['value'])
    for _, metric, value in health_checks
    if metric['name'] == 'ybp_health_check_client_cert_validity_days'
}
    
print(f"n2n: {n2n_validity_days}\nc2n: {c2n_validity_days}") 